In [ ]:
!pip install hyperas

In [2]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from tensorflow.keras.constraints import max_norm
from random import randint
import keras
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import os
import matplotlib.pyplot as plt

In [3]:
os.chdir('/content/drive/My Drive/github/NNanobody/regression')

In [4]:
model = keras.models.load_model('./seq_32_32')

Everything up-to-date


In [ ]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    # DROPOUT = 0.3
    
    model = Sequential()
    model.add(Flatten(input_shape=(20, 1, 20)))
    # model.add(Dense(32,activation='relu',kernel_constraint=
    (W_maxnorm), kernel_regularizer='l1'))
    model.add(Dense(32,activation='relu',kernel_constraint=max_norm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    # model.add(Dense(32,activation='relu',kernel_constraint=max_norm(W_maxnorm), kernel_regularizer='l1'))
    model.add(Dense(32,activation='relu',kernel_constraint=max_norm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    #model.add(Dense(2,W_constraint=maxnorm(W_maxnorm)))
    #model.add(Activation('softmax'))
    model.add(Dense(1))

    myoptimizer = RMSprop(lr={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-6)
    # myoptimizer = RMSprop(learning_rate=1e-1, rho=0.9, epsilon=1e-06)
    #mylossfunc = 'binary_crossentropy'
    mylossfunc='mean_squared_error'#, optimizer=myoptimizer,metrics=['accuracy'])
    model.compile(loss=mylossfunc, optimizer=myoptimizer)
    result = model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1, verbose=False)

    val_loss = np.amax(result.history['val_loss'])
    # model_arch = 'MODEL_ARCH'
    # bestaccfile = join('TOPDIR',model_arch,model_arch+'_hyperbestacc')
    # reportAcc(acc,score,bestaccfile)

    return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [ ]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test_new.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32_32',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 10/10 [06:43<00:00, 40.31s/it, best loss: 0.28897520899772644]


In [ ]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1,verbose=True)

Epoch 1/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3378 - val_loss: 0.3084
Epoch 2/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3337 - val_loss: 0.3051
Epoch 3/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3325 - val_loss: 0.3040
Epoch 4/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3300 - val_loss: 0.3026
Epoch 5/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3261 - val_loss: 0.3008
Epoch 6/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3256 - val_loss: 0.3007
Epoch 7/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3243 - val_loss: 0.2999
Epoch 8/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3246 - val_loss: 0.2992
Epoch 9/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3214 - val_loss: 0.2983
Epoch 10/20
571/571 [==============================] - 2s 3ms/step - loss: 0.3200 - val_loss: 0.2977

In [ ]:
best_model.evaluate(X_test, Y_test)

710/710 [==============================] - 1s 2ms/step - loss: 0.3190


0.3190174996852875

In [ ]:
best_model.save('./seq_32_32')

INFO:tensorflow:Assets written to: ./seq_32_32/assets


In [ ]:
model = keras.models.load_model('./seq_32_32')

In [ ]:
X_train, Y_train, X_test, Y_test = train_data()
model.evaluate(X_test, Y_test)

710/710 [==============================] - 3s 2ms/step - loss: 0.3190


0.3190174996852875